# Data cleaning and completion for table: GeneralData

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import sys

sys.path.append('../')

In [2]:
os.getcwd()

'/home/michael/Projects/DSlab2/data-science2-lab-final-project/src/team_4'

In [3]:
# Load the provided Excel file
# The data is present in the data folder two directories up
data_dir = os.path.abspath(os.path.join(os.getcwd(),"..","data"))
dataset_path = os.path.join(data_dir, "rehospitalization.xlsx")
OutPutDate = os.path.join(data_dir, 'GeneralData.csv')

In [4]:
# Load the 'GeneralData' sheet into a DataFrame for cleaning and completion
general_data_df = pd.read_excel(dataset_path, sheet_name='GeneralData')

# Display the first few rows to understand the structure and identify potential issues
general_data_df.head()

,Patient,age,Gender,גורם משלם,משקל,גובה,BMI,מחלות כרוניות,השכלה,מספר ילדים,מצב משפחתי,תרופות קבועות
0,1043772,84,זכר,כללית,92.0,170.0,34.2,1,ללא,3,נשוי,"10815 , 11282 , 1183 , 1196 , 12758 , 13027 ,..."
1,77493,89,נקבה,‎ממון עצמי-לא מב‎,45.0,150.0,21.4,0,NaN,1,גרוש,"10337 , 1130 , 1448 , 1466 , 186 , 2482 , 290..."
2,37680,87,זכר,‎ממון עצמי-לא מב‎,67.7,165.0,24.9,0,פנסיונר,2,נשוי,"1183 , 1310 , 2451 , 2566 , 4967 , 6718 , 672..."
3,70066,90,נקבה,מאוחדת,94.0,165.0,31.6,0,NaN,1,גרוש,"1100 , 1181 , 1182 , 1183 , 12757 , 1283 , 12..."
4,13616,87,זכר,כללית,70.0,163.0,26.3,0,NaN,3,נשוי,"10214 , 1212 , 12725 , 2882 , 6737 , 7852 , 7..."


In [5]:
def summary(df):
    nonnans = df.shape[0] - df.isna().sum()
    nonnansPrc = (nonnans / df.shape[0] * 100).apply("{0:.2f}%".format)
    sam = df.sample(1, random_state=42).squeeze()
    res = pd.DataFrame([sam.index, df.dtypes.astype(str), nonnans,
                        nonnansPrc, df.nunique(), sam]).transpose()
    res.columns = ["Column", "data type", "non-null values", 
                   "non-null values %", "unique values", "example"]
    res.sort_values("unique values",ascending=False, inplace=True)
    return res

In [6]:
summary(general_data_df)

,Column,data type,non-null values,non-null values %,unique values,example
0,Patient,int64,4535,100.00%,4535,144443
11,תרופות קבועות,object,4495,99.12%,4490,"1183 , 2213 , 5913"
8,השכלה,object,2641,58.24%,648,NaN
6,BMI,float64,3504,77.27%,308,28.3
4,משקל,float64,3718,81.98%,177,80.0
5,גובה,float64,3698,81.54%,83,168.0
1,age,int64,4535,100.00%,42,98
9,מספר ילדים,object,4373,96.43%,21,2
3,גורם משלם,object,4524,99.76%,7,‎ממון עצמי-לא מב‎
10,מצב משפחתי,object,4474,98.65%,5,נשוי


## Education  
education column contains a lot of free form text and requires special treatment.


In [7]:
general_data_df.rename(columns={"השכלה": "education"},inplace=True)
general_data_df.education = general_data_df.education.fillna("")
general_data_df.education = general_data_df.education.astype(str)

In [8]:
import re
def spl(n):
    if n!=None:
        return re.split(', |\. | |,|\.|\\|- |-|\/|\+|\(|\)', str(n))
    else:
        return [""]

def token_analysis(col,sortby = "cnt", ascending=False):
    #"token"
    names = col.apply(func=spl).values.tolist()
    #print(names)
    words = sum(names,[])
    
    unique, counts = np.unique(words, return_counts=True)
    wc = pd.DataFrame({"token":unique, "cnt": counts})
    
    tokens = wc.sort_values(sortby,ascending=ascending).reset_index(drop=True)
    return tokens

First, let's see what kind of words can we see in the column  
Analyze frequencies of all unique words in the education column

In [9]:
ta = token_analysis(general_data_df['education'])
print(ta.shape)
ta.head(20)

(643, 2)


,token,cnt
0,,2053
1,פנסיונר,595
2,פנסיונרית,403
3,לא,269
4,פנסיה,165
5,עובד,160
6,עובדת,143
7,גימלאית,88
8,בית,84
9,גימלאי,75


In [10]:
#ta.to_excel("tokens.xlsx")

### Manual labor
We observe a total of 644 unique words in the entire education column. 
Not a huge amount, so can be manually separated into 5 groups:

 - Pension: all the words indicating the persion is a pensioner. due to manual manner of input and human mistake, there are 56 variations of that word. examles: `גמלאית גמלאי פנסיוניר פנסיונרת`
  - Role: the employment role that a person had before, ex: `מנהל  אחות  מהנדס  חקלאי  רופא`
  - Assignment: more granular description of person's employment, ex: `מסחר  מכניקה מזגנים  נהיגה`
  - Status: words indicating person's disabled status, ex: `נכה  שוהה  שכלית מצבו`
  - Negation: variants of "No", mostly used to indicate the person is not working. ex: `רלוונטי  ללא  אינה`
  - Junk: words that do not bear significance for the research at hand, such as prepositions, temporal indicators, given names, etc. ex: `היה  כרגע  כיום  היתה  במקצוע  רפתי בלוונשטיין`
  
The separation is done by moving a word into appropriate column in an excel document. The final version is provided in the tokens_dict.xlsx file


### text patterns in the column

once the classification of words is complete, we can use the excel file as our token database to classify the descritions in 'education' column


In [11]:
tokdic = pd.read_excel("tokens_dict.xlsx")

In [12]:
# eliminate null values from tokens_dict.xlsx that appeared as a result of manually moving cells in excel document
for col in tokdic.columns:
    tmp = tokdic[col][~tokdic[col].isna()].reset_index(drop=True)
    tokdic[col] = tmp.sort_values().reset_index(drop=True)
tokdic.dropna(axis = 0, how="all")

,pension,pension_n,assignment,assignment_n,role,role_n,status,status_n,negation,negation_n,junk,junk_n
0,בגמלאות,1.0,Jשמל,1.0,אב,1.0,בשבץ,1.0,אין,1.0,אגד,1.0
1,בפנסיה,1.0,אבטחה,1.0,אגרונום,1.0,חולה,1.0,אינה,10.0,איש,1.0
2,גימאלית,1.0,אדמניסטרטיבי,1.0,אומן,1.0,מצבו,1.0,אינו,13.0,אתו,1.0
3,גימלאי,1.0,אוויר,1.0,אונקולוג,1.0,נכה,1.0,לא\s?,15.0,בארקיע,1.0
4,גימלאית,1.0,אווירית,1.0,אחות,1.0,נכות,1.0,ללא,22.0,בבעברו,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
313,NaN,NaN,תעשיה,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
314,NaN,NaN,תעשייה,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
315,NaN,NaN,תעשייתי,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
316,NaN,NaN,תפירה,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
symbols = {'pension': 'P',     
           'assignment': 'A',    
           'role': 'R',       
           'status': 'S',  
           'negation': 'N',
           'junk': 'J'}

In [14]:
# extra len column for sorting the tokens by their length
# required to prevent shorter words to be extracted before longer ones
# e.g: 'fortitude' should be extracted before 'for', otherwise  we'll end up with 'titude' 
tokendb = tokdic.loc[:,list(symbols.keys())]
tokendb.loc[:,'len'] = tokendb['pension'].str.len()

We will first try to identify whether there are standard patterns of descriptions in this column
By replacing each word with a symbol that indicate it's group, we can reduce all the descriptions into a limited amount of templates.
  

In [15]:
def research(text, patt, grp=0):       
    result = re.search(patt, text)
    return result.group(grp) if result else None

def construct_pattern(col):
    tokendb.loc[:,'len'] = tokendb[col].str.len()
    tdb = tokendb.sort_values(['len',col], ascending=False)
    patt_col = tdb.loc[~tdb[col].isna(),col]
    patt_col = [re.escape(patt) for patt in patt_col]
    pattern = "|".join(patt_col)
    return pattern

pattern = []
syms = []

pattern.append('\d+([,.]\d+)?') 
syms.append('J')
for tok, sym in symbols.items():    
    #tokens = 'pension'
    pattern.append(construct_pattern(tok))
    syms.append(sym) 
     
pattern.append(r'(, |\. |,|\.|\\|- |-|\/|\+|\(|\))+')
syms.append(' ')



def split_name2(text):

    templ = str(text)        
    status=[]
    role = []
    assgn = []
    job = []
    for pat, sym in zip(pattern, syms):
        found = re.findall(pat, templ)
        if found:
            if sym == 'P':
                status.append("pension")
            if sym == 'N':
                status.append("not working")
            if sym == 'S':
                status.append("disabled")
            if sym == 'R':
                job.append(found)
            if sym == 'A':
                job.append(found)
                
        templ = re.sub(pat, sym, templ).strip()
    job = sum(job,[])
    job.reverse()
    job = " ".join(job)
    status = " ".join(status)
    return {"template": templ, "status": status,"job": job}


In [16]:
data = general_data_df[["education"]].dropna()# .loc[:1000]
data

,education
0,ללא
1,
2,פנסיונר
3,
4,
...,...
4530,אלקטרונאי בפנסיה
4531,
4532,פנסיונר
4533,תיווך


In [17]:
applied_df = data.apply(lambda row: split_name2(row.education), axis='columns', result_type='expand')
df = pd.concat([data, applied_df], axis='columns')

In [18]:
df[~df.job.isna()]

,education,template,status,job
0,ללא,N,not working,
1,,,,
2,פנסיונר,P,pension,
3,,,,
4,,,,
...,...,...,...,...
4530,אלקטרונאי בפנסיה,R P,pension,אלקטרונאי
4531,,,,
4532,פנסיונר,P,pension,
4533,תיווך,A,,תיווך


However, upon examination, this technique proved to be not robust enough.

### N-grams
Since the content in education is a free-form text, the order and connection between words is important. 
N-grams is a simple, yet powerful technique for this kind of problem. 


We will use 2 and 3 -grams to identify word combinations that are frequent in the dataset, hopefully helping us to extract meaningful and concese dscription of person' employment.


before extracting ngrams, we clear the input from garbage, such as punctuation, numbers and junk words.
Also we extract status by searching for any instances of 'pension', 'not working' or 'disabled'

In [19]:
def extract_clear(text):
    text = str(text)
    text = re.sub('\d+([,.]\d+)?', "", text).strip()
    text = re.sub(r'"|(, |\. |,|\.|\\|- |-|\/|\+|\(|\))+', "", text).strip()
    words = text.split(" ")
    job = []    
    status=""  

    for word in words:          
        if word in tokendb.negation.values :
            status = "not working"
            continue
        if word in tokendb.pension.values  :
            status = "pension"
            continue           
        if word in tokendb.status.values  :
            status = "disabled"
            continue
        if word in tokendb.assignment.values  :
            job.append(word)
            continue
        if word in tokendb.role.values  :
            job.append(word)
            continue       

    job = " ".join(job) # recombine content that remains after cleaning
    return {"status": status,"job": job}


In [20]:
edu_data = general_data_df[["education"]].dropna()

In [21]:
applied_df = edu_data.apply(lambda row: extract_clear(row.education), axis='columns', result_type='expand')
edu_data = pd.concat([edu_data, applied_df], axis='columns')
edu_data.head()

,education,status,job
0,ללא,not working,
1,,,
2,פנסיונר,pension,
3,,,
4,,,


In [22]:
import nltk

In [23]:
nltk.download('punkt_tab',)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/michael/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [24]:
def ngrams(item, n=2):
    item = str(item)
    patt = r'(, |\. |,|\.|\\|- |-|\/|\+|\(|\))+'
    item = re.sub(patt, " ", item)
    tokens = nltk.word_tokenize(item)
    return list(nltk.ngrams(tokens, n))

In [25]:
# extract all bigrams and trigrams from the entire job column
bigrams = sum(edu_data.job.apply(ngrams, n=2),[])
trigrams = sum(edu_data.job.apply(ngrams, n=3),[])

In [26]:
# for every bi and trigram, calculate it's frequency
BigramFD = nltk.FreqDist(bigrams)
TrigramFD = nltk.FreqDist(trigrams)

In [27]:
# for a given description, identify which bi or trigram is the most likely to describe the job of the person
# This is based on an assumption that common job titles will appear more frequently in the entire dataset than 
# unrelated text
# additionally, to equalize the chances between the bi and trigrams, we multiply the freq by 2 or 3 respectively 

def best_ngram(job):
    bigrams = ngrams(job,n=2)
    if bigrams==[]:
        return job
    bgs = {bg: BigramFD[bg]*2 for bg in bigrams}
    
    trigrams = ngrams(job,n=3)
    tgs = {tg: TrigramFD[tg]*3 for tg in trigrams}
    bgs.update(tgs)

    gram = max(bgs, key=bgs.get)
    return " ".join(gram)

In [28]:
applied_df = edu_data.apply(lambda row: best_ngram(row.job), axis='columns', result_type='expand')
edu_data["job"] = applied_df

In [29]:
test = edu_data[edu_data.education.str.contains(" ")]
test[:50]

,education,status,job
5,פנסיה. לפני בטקסטיל.,,בטקסטיל
9,עסק פרטי - בעלת חנות,,עסק פרטי
20,יועץ לאדריכלים,,יועץ לאדריכלים
32,לא עובדת,,
33,לא עובדת גימלאית .,pension,
51,חינוך מיוחד,,חינוך מיוחד
55,נהג אוטובוס,,נהג אוטובוס
57,עצמאי מכניקה,,עצמאי מכניקה
59,"גימלאי, עבד כמסגר",,כמסגר
77,לא עובד,,


We will store this data separately for now, as otherwise it will complicate the translation part.   

Below are the most common job titles we found:

In [30]:
unique, counts = np.unique(edu_data.job, return_counts=True)
wc = pd.DataFrame({"token":unique, "cnt": counts})

alljobs = wc.sort_values("cnt",ascending=False).reset_index(drop=True)
print(alljobs.shape)
alljobs[:50]

(409, 2)


,token,cnt
0,,3773
1,עקרת בית,68
2,אחות,24
3,מורה,24
4,חקלאי,14
5,מהנדס,14
6,עצמאי,13
7,נהג מונית,11
8,נהג,10
9,רופא,10


In [31]:
#alljobs.to_csv("alljobs.csv", index = False)

### Categorization

At this point we've managed to reduce trhe amount of classes of jobs to 400.
This is still way too much for a classification algorithm.  

Hence we point to modern AI technology to perform the final step.
We request ChatGPT to map these 400 jobs into couple dozen broad categories.
We are able to reduce the amount of categories to 30 
The result is very good, it is provided in file jobs_mapping.csv

In [32]:
jm = pd.read_csv("jobs_mapping.csv") #, na_filter='NaN' )


In [33]:
# inject mapping for empty value
e = pd.DataFrame({"token": [""], "mapping": ["Unknown"]}, index=[len(jm)])
jm = pd.concat([jm,e],axis=0)

In [34]:
# validate that all the mappings exist in edu_data
my = set(edu_data.job.values)

test = [a for a in jm.token.values if a in my]

len(test)

409

This is a summary of how many jobs each broad category maps to 

In [35]:
from pandasql import  sqldf
q = """
select mapping, count(token) as jobs
from jm
group by mapping 
order by jobs desc

"""
wat = sqldf(q)
wat
#print("\n".join(wat.mapping))

,mapping,jobs
0,Healthcare,49
1,Skilled Trades,40
2,Education,36
3,Retail,34
4,Finance,23
5,Government,21
6,Administration,20
7,Construction,17
8,Transportation,15
9,Engineering,14


In [36]:
jmd = {item[1]["token"]: item[1]["mapping"] for item in jm.iterrows() }

In [37]:
edu_data["jobcat"] = edu_data.job.map(jmd, na_action='ignore')


In [38]:
edu_data[edu_data.jobcat.isna()]

,education,status,job,jobcat


In [39]:
#edu_data.to_excel("temp.xlsx")

## Translate hebrew content into english 

Hebrew text, being an RTL language, presents a great challenge when writing code, hence we want to translate it.
Thankfully, by omitting education column, we remain with a very limited set of values to translate.
We use Team 11 approach and code for this.

In [40]:
del general_data_df["education"]

In [41]:
from  team_11.task7.task7 import translate_dataframe, extract_hebrew_content, contains_hebrew

In [42]:
content = extract_hebrew_content(general_data_df)

In [43]:
content

{'columns': ['גורם משלם',
  'משקל',
  'גובה',
  'מחלות כרוניות',
  'מספר ילדים',
  'מצב משפחתי',
  'תרופות קבועות'],
 'content': ['זכר',
  'נקבה',
  'כללית',
  '\u200eממון עצמי-לא מב\u200e',
  'מאוחדת',
  'מכבי',
  'לאומית',
  '\u200eשרות בתי הסוהר\u200e',
  '\u200eתייר/תעריף מלא\u200e',
  '1 + 2 (נפטרו)',
  '2( ילדים בחוץ לארץ)',
  'נכדה  אחת',
  'אין',
  'נשוי',
  'גרוש',
  'אלמן',
  'רווק',
  'פרוד']}

In [44]:
c = content['columns'] + content['content'] 
c

['גורם משלם',
 'משקל',
 'גובה',
 'מחלות כרוניות',
 'מספר ילדים',
 'מצב משפחתי',
 'תרופות קבועות',
 'זכר',
 'נקבה',
 'כללית',
 '\u200eממון עצמי-לא מב\u200e',
 'מאוחדת',
 'מכבי',
 'לאומית',
 '\u200eשרות בתי הסוהר\u200e',
 '\u200eתייר/תעריף מלא\u200e',
 '1 + 2 (נפטרו)',
 '2( ילדים בחוץ לארץ)',
 'נכדה  אחת',
 'אין',
 'נשוי',
 'גרוש',
 'אלמן',
 'רווק',
 'פרוד']

In [45]:
# Manual labor: 
# take output of c, put it into google translate, paste the translated text as t:
# correct any mistakes or redundant values if necessary
t = ['Paying factor', 'Weight', 'Height', 'Chronic diseases',
     'Number of children', 'Marital status', 'Regular medications',
     'Male', 'Female', 'General', 'Self-financed Not from', 'Mauhedat',
     'Maccabi', 'National', 'Prison services', 'Tourist/full rate', '1 + 2 (deceased)',
     '2 ( children abroad)', 'one granddaughter', 'none', 'married', 'divorced', 
     'widowed', 'single', 'separated']

In [46]:
# copy-paste the output of d as translate_dict
d ={k:v for k,v in zip(c,t)}
d

{'גורם משלם': 'Paying factor',
 'משקל': 'Weight',
 'גובה': 'Height',
 'מחלות כרוניות': 'Chronic diseases',
 'מספר ילדים': 'Number of children',
 'מצב משפחתי': 'Marital status',
 'תרופות קבועות': 'Regular medications',
 'זכר': 'Male',
 'נקבה': 'Female',
 'כללית': 'General',
 '\u200eממון עצמי-לא מב\u200e': 'Self-financed Not from',
 'מאוחדת': 'Mauhedat',
 'מכבי': 'Maccabi',
 'לאומית': 'National',
 '\u200eשרות בתי הסוהר\u200e': 'Prison services',
 '\u200eתייר/תעריף מלא\u200e': 'Tourist/full rate',
 '1 + 2 (נפטרו)': '1 + 2 (deceased)',
 '2( ילדים בחוץ לארץ)': '2 ( children abroad)',
 'נכדה  אחת': 'one granddaughter',
 'אין': 'none',
 'נשוי': 'married',
 'גרוש': 'divorced',
 'אלמן': 'widowed',
 'רווק': 'single',
 'פרוד': 'separated'}

In [47]:
translate_dict= {'גורם משלם': 'Paying agent',
 'משקל': 'Weight',
 'גובה': 'Height',
 'מחלות כרוניות': 'Chronic diseases',
 'מספר ילדים': 'Number of children',
 'מצב משפחתי': 'Marital status',
 'תרופות קבועות': 'Regular medications',
 'זכר': 'Male',
 'נקבה': 'Female',
 'כללית': 'Clalit',
 'ממון עצמי-לא מב': 'Self-financed',
 'מאוחדת': 'Meuhedet',
 'מכבי': 'Maccabi',
 'לאומית': 'Leumit',
 'שרות בתי הסוהר': 'Prison services',
 'תייר/תעריף מלא': 'Tourist/full rate',
 "1 2 \u05e0\u05e4\u05d8\u05e8\u05d5": "3",
 "2 \u05d9\u05dc\u05d3\u05d9\u05dd \u05d1\u05d7\u05d5\u05e5 \u05dc\u05d0\u05e8\u05e5": "2",
 "\u05e0\u05db\u05d3\u05d4 \u05d0\u05d7\u05ea": "1",
 'אין': 'none',
 'נשוי': 'married',
 'גרוש': 'divorced',
 'אלמן': 'widowed',
 'רווק': 'single',
 'פרוד': 'separated'}

In [48]:
general_data_df = translate_dataframe(general_data_df, translate_dict )

In [49]:
# validate that all the hebrew text was translated.
# the columns and content are supposed to be empty.
# if not, that's probably due to special characters 
check = extract_hebrew_content(general_data_df)
check

{'columns': [], 'content': []}

In [50]:
general_data_df

,Patient,age,Gender,Paying agent,Weight,Height,BMI,Chronic diseases,Number of children,Marital status,Regular medications
0,1043772,84,Male,Clalit,92.0,170.0,34.2,1,3,married,"10815, 11282, 1183, 1196, 12758, 13027, 1333, ..."
1,77493,89,Female,Self-financed,45.0,150.0,21.4,0,1,divorced,"10337, 1130, 1448, 1466, 186, 2482, 2907, 3261..."
2,37680,87,Male,Self-financed,67.7,165.0,24.9,0,2,married,"1183, 1310, 2451, 2566, 4967, 6718, 6720, 6737..."
3,70066,90,Female,Meuhedet,94.0,165.0,31.6,0,1,divorced,"1100, 1181, 1182, 1183, 12757, 1283, 12946, 13..."
4,13616,87,Male,Clalit,70.0,163.0,26.3,0,3,married,"10214, 1212, 12725, 2882, 6737, 7852, 7999, 92..."
...,...,...,...,...,...,...,...,...,...,...,...
4530,5473,82,Male,Self-financed,69.0,180.0,21.3,0,3,married,"1150, 1682, 2043, 2150, 2188, 2566, 2606, 2897..."
4531,980451,79,Male,Maccabi,64.0,170.0,22.1,0,4,married,NaN
4532,430555,99,Male,Clalit,60.0,170.0,20.8,0,3,widowed,"12284, 2384, 2412, 2791, 2843, 3211, 3298, 347..."
4533,737553,75,Male,Maccabi,97.0,170.0,33.6,0,3,married,"1183, 12730, 1637, 2568, 37, 3724, 4856, 6432,..."


In [51]:
# Check for missing data
# Get basic statistics to identify potential outliers and data ranges
summary(general_data_df)

,Column,data type,non-null values,non-null values %,unique values,example
0,Patient,int64,4535,100.00%,4535,144443
10,Regular medications,object,4495,99.12%,4490,"1183, 2213, 5913"
6,BMI,float64,3504,77.27%,308,28.3
4,Weight,float64,3718,81.98%,177,80.0
5,Height,float64,3698,81.54%,83,168.0
1,age,int64,4535,100.00%,42,98
8,Number of children,object,4373,96.43%,21,2
3,Paying agent,object,4524,99.76%,7,Self-financed
9,Marital status,object,4474,98.65%,5,married
2,Gender,object,4535,100.00%,2,Male


In [52]:
pd.DataFrame(general_data_df.describe()).transpose()

,count,mean,std,min,25%,50%,75%,max
Patient,4535.0,360635.420507,325399.296102,31.00,87996.50,241523.0,602290.0,1171452.0
age,4535.0,80.147078,8.640097,65.00,73.00,80.0,87.0,111.0
Weight,3718.0,75.901668,147.942371,30.00,61.85,72.0,83.0,9036.8
Height,3698.0,165.239313,29.634842,1.58,160.00,165.0,170.0,1765.0
BMI,3504.0,1229.321946,29376.471228,3.20,23.40,26.3,30.0,830000.0
Chronic diseases,4535.0,0.137155,0.344049,0.00,0.00,0.0,0.0,1.0


In [53]:
# Identify outliers in 'BMI', 'weight', and 'height'
# For BMI, assume normal human range is 10 to 60
bmi_outliers = general_data_df[(general_data_df['BMI'] < 10) | (general_data_df['BMI'] > 60)]

# For weight, assume normal human range is 30 to 300 kg
weight_outliers = general_data_df[(general_data_df['Weight'] < 30) | (general_data_df['Weight'] > 300)]

# For height, assume normal human range is 140 to 210 cm
height_outliers = general_data_df[(general_data_df['Height'] < 140) | (general_data_df['Height'] > 210)]

bmi_outliers, weight_outliers, height_outliers

(      Patient  age  Gender   Paying agent  Weight  Height       BMI  \
 137   1132829   82  Female         Clalit    57.5    57.5     492.5   
 348    112253   82    Male         Clalit    88.0    80.0     137.5   
 658     51497   94    Male  Self-financed   165.0    60.0     458.3   
 664    184567   91  Female  Self-financed    64.0   450.0       3.2   
 1114   823081   93    Male        Maccabi    80.0    48.0     694.4   
 1268    53726   85  Female       Meuhedet    60.0    58.0     178.4   
 1285   178335   87  Female        Maccabi    63.0   200.0  630000.0   
 1448   128998   73    Male         Clalit    75.0   172.0  750000.0   
 1620   755744   81    Male         Clalit    80.0   178.0      69.9   
 2233   172354   78    Male         Clalit    83.0   170.0  830000.0   
 2488   205368   80    Male         Clalit    56.0    65.0     132.5   
 2671   289488   72    Male         Clalit    74.0    74.0     319.6   
 2700   197599   70    Male  Self-financed    70.0   165.0  7000

In [54]:
# Replace outliers in BMI, weight, and height with NaN
general_data_df.loc[(general_data_df['BMI'] < 10) | (general_data_df['BMI'] > 60), 'BMI'] = np.nan
general_data_df.loc[(general_data_df['Weight'] < 30) | (general_data_df['Weight'] > 300), 'Weight'] = np.nan
general_data_df.loc[(general_data_df['Height'] < 140) | (general_data_df['Height'] > 210), 'Height'] = np.nan

# Recalculate BMI for entries where weight and height are available
general_data_df['BMI'] = general_data_df['Weight'] / ((general_data_df['Height'] / 100) ** 2)

# Display summary of changes
general_data_df[['BMI', 'Weight', 'Height']].describe()

,BMI,Weight,Height
count,3655.000000,3717.00000,3662.000000
mean,26.688510,73.49088,165.584926
std,5.334254,16.68483,8.653187
min,13.064305,30.00000,140.000000
25%,23.183391,61.80000,160.000000
50%,25.951557,72.00000,165.000000
75%,29.386256,83.00000,170.000000
max,78.125000,200.00000,200.000000


In [55]:

# Filling missing values for categorical data using mode
general_data_df['Paying agent'].fillna(general_data_df['Paying agent'].mode()[0], inplace=True)
#general_data_df['השכלה'].fillna(general_data_df['השכלה'].mode()[0], inplace=True)
general_data_df['Marital status'].fillna(general_data_df['Marital status'].mode()[0], inplace=True)

# Filling missing values for numeric data using median
general_data_df['Weight'].fillna(general_data_df['Weight'].median(), inplace=True)
general_data_df['Height'].fillna(general_data_df['Height'].median(), inplace=True)
general_data_df['BMI'].fillna(general_data_df['BMI'].median(), inplace=True)
#general_data_df['Number of children'].fillna(general_data_df['Number of children'].median(), inplace=True)

# Checking if there are any remaining missing values
summary(general_data_df)

/tmp/ipykernel_2006413/4243214794.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  general_data_df['Paying agent'].fillna(general_data_df['Paying agent'].mode()[0], inplace=True)
/tmp/ipykernel_2006413/4243214794.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, w

,Column,data type,non-null values,non-null values %,unique values,example
0,Patient,int64,4535,100.00%,4535,144443
10,Regular medications,object,4495,99.12%,4490,"1183, 2213, 5913"
6,BMI,float64,4535,100.00%,1257,28.344671
4,Weight,float64,4535,100.00%,176,80.0
5,Height,float64,4535,100.00%,55,168.0
1,age,int64,4535,100.00%,42,98
8,Number of children,object,4373,96.43%,21,2
3,Paying agent,object,4535,100.00%,7,Self-financed
9,Marital status,object,4535,100.00%,5,married
2,Gender,object,4535,100.00%,2,Male


In [56]:
general_data_df['Number of children'].unique()

array([3, 1, 2, 4, 0, 7, 5, 8, 6, nan, 10, 9, '3', 11, '4 4', 12, '2', 13,
       15, 14, '1', 'none'], dtype=object)

In [57]:
# Attempt to clean 'מספר ילדים' column by converting entries to numeric values
# We'll extract the first number before any non-numeric characters

import re

def extract_numeric(value):
    if isinstance(value, str):
        # Extract the first number that appears in the string
        match = re.search(r'\d+', value)
        if match:
            return int(match.group(0))
        else:
            return np.nan
    return value

# Apply the function to clean the 'מספר ילדים' column
general_data_df['Number of children'] = general_data_df['Number of children'].apply(extract_numeric)

# Now try filling missing values with the median
general_data_df['Number of children'].fillna(general_data_df['Number of children'].median(), inplace=True)

# Check the column to ensure it's clean and filled
general_data_df['Number of children'].describe()


/tmp/ipykernel_2006413/3294783957.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  general_data_df['Number of children'].fillna(general_data_df['Number of children'].median(), inplace=True)


count    4535.000000
mean        3.272326
std         2.062258
min         0.000000
25%         2.000000
50%         3.000000
75%         4.000000
max        15.000000
Name: Number of children, dtype: float64

In [58]:
general_data_df["status"] =  edu_data["status"]
general_data_df["job"] =  edu_data["job"]
general_data_df["jobcat"] =  edu_data["jobcat"]
general_data_df

,Patient,age,Gender,Paying agent,Weight,Height,BMI,Chronic diseases,Number of children,Marital status,Regular medications,status,job,jobcat
0,1043772,84,Male,Clalit,92.0,170.0,31.833910,1,3.0,married,"10815, 11282, 1183, 1196, 12758, 13027, 1333, ...",not working,,Unknown
1,77493,89,Female,Self-financed,45.0,150.0,20.000000,0,1.0,divorced,"10337, 1130, 1448, 1466, 186, 2482, 2907, 3261...",,,Unknown
2,37680,87,Male,Self-financed,67.7,165.0,24.866850,0,2.0,married,"1183, 1310, 2451, 2566, 4967, 6718, 6720, 6737...",pension,,Unknown
3,70066,90,Female,Meuhedet,94.0,165.0,34.527089,0,1.0,divorced,"1100, 1181, 1182, 1183, 12757, 1283, 12946, 13...",,,Unknown
4,13616,87,Male,Clalit,70.0,163.0,26.346494,0,3.0,married,"10214, 1212, 12725, 2882, 6737, 7852, 7999, 92...",,,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4530,5473,82,Male,Self-financed,69.0,180.0,21.296296,0,3.0,married,"1150, 1682, 2043, 2150, 2188, 2566, 2606, 2897...",pension,אלקטרונאי,Technology
4531,980451,79,Male,Maccabi,64.0,170.0,22.145329,0,4.0,married,NaN,,,Unknown
4532,430555,99,Male,Clalit,60.0,170.0,20.761246,0,3.0,widowed,"12284, 2384, 2412, 2791, 2843, 3211, 3298, 347...",pension,,Unknown
4533,737553,75,Male,Maccabi,97.0,170.0,33.564014,0,3.0,married,"1183, 12730, 1637, 2568, 37, 3724, 4856, 6432,...",,תיווך,Real Estate


In [59]:
# Fill missing values in 'תרופות קבועות' using the mode of the column
general_data_df['Regular medications'].fillna(general_data_df['Regular medications'].mode()[0], inplace=True)

# Verify that there are no remaining missing values
summary(general_data_df)

/tmp/ipykernel_2006413/1460784401.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  general_data_df['Regular medications'].fillna(general_data_df['Regular medications'].mode()[0], inplace=True)


,Column,data type,non-null values,non-null values %,unique values,example
0,Patient,int64,4535,100.00%,4535,144443
10,Regular medications,object,4535,100.00%,4490,"1183, 2213, 5913"
6,BMI,float64,4535,100.00%,1257,28.344671
12,job,object,4535,100.00%,409,
4,Weight,float64,4535,100.00%,176,80.0
5,Height,float64,4535,100.00%,55,168.0
1,age,int64,4535,100.00%,42,98
13,jobcat,object,4535,100.00%,30,Unknown
8,Number of children,float64,4535,100.00%,16,2.0
3,Paying agent,object,4535,100.00%,7,Self-financed


In [60]:
# Save the updated dataset to a new Excel File
general_data_df.to_excel(OutPutDate, index=False)

In [61]:
from datetime import datetime
print("last full run: ", datetime.now().strftime("%Y.%m.%d-%H.%M.%S"))

last full run:  2024.09.09-19.20.17
